La finalidad de este segundo cuaderno es hacerle finetuning a un modelo basado en bert, entrenado en español (bert-base-spanish) para predecir la "dificultad" de un prompt que se le hace a nuestra herramienta "Chatbot CV". Así como si la pregunta no está relacionada con la informacion del CV.

De esta forma, se optimizan costes al usar LLMs más caros (e inteligentes) unicamente cuando es necesario. Así como no llamar a ningun LLM y ofrecer una respuesta estática en caso de que se utilice para otros fines.

## 1. Importar lo necesario

Librerias necesarias etc.

In [1]:
# Celda 1: Importar dependencias
import pandas as pd
import numpy as np
import torch
import re
import copy
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import unicodedata

## 2. Generar dataset
Lo definiremos aqui mismo. Los valores de "dificultad" serán:

0 -> Pregunta "fácil" - LLM Barato

1 -> Pregunta "dificil" - LLM Inteligente

2 -> Pregunta no relacionada - Bloquear

Ejemplos de preguntas serían:
* Facil: "¿qué ha estudiado Marcos?"
* Dificil: "¿Sería capaz Marcos de trabajar en un puesto que tenga las tecnologías [...]?"
* No relacionado: "dame una receta de lasaña".

In [2]:
# Preguntas fáciles (dificultad 0) - Información DIRECTA del CV
prompts_diff0 = [
    # Educación y certificaciones - información factual
    "¿Qué estudios tiene Marcos?",
    "¿Dónde estudió Marcos?",
    "¿Qué certificaciones posee Marcos?",
    "¿En qué año se graduó Marcos?",
    "¿Qué nivel de inglés tiene según su CV?",
    "¿Qué otros idiomas habla?",
    "Formación académica de Marcos",
    "¿Cuándo hizo su máster en IA?",
    "Títulos universitarios",
    "¿Qué certificado de Python tiene?",

    # Experiencia laboral - datos concretos
    "¿En qué empresas ha trabajado?",
    "¿Cuánto tiempo trabajó en Outlier?",
    "¿Qué puesto ocupó en Grupo Oro?",
    "Fechas de sus prácticas profesionales",
    "Puesto actual según CV",
    "Experiencia total en años",
    "¿Dónde realizó sus prácticas?",
    "Fecha de inicio como LLM Specialist",
    "Primer trabajo profesional",
    "Cargo actual",

    # Habilidades y tecnologías - listado directo
    "Lenguajes de programación que conoce",
    "Herramientas de desarrollo que usa",
    "¿Sabe usar WordPress?",
    "Frameworks de IA mencionados",
    "¿Tiene experiencia con Python?",
    "Bases de datos que maneja",
    "Bibliotecas científicas que conoce",
    "¿Tiene conocimientos de cloud?",
    "Tecnologías ML en su CV",
    "¿Conoce sobre ciberseguridad?",

    # Proyectos - información básica
    "¿Qué es el proyecto ChatCV?",
    "¿Qué es Gather-Tracker?",
    "Tecnologías usadas en ChatCV",
    "¿Cuándo creó Fox-Detector?",
    "¿Para qué sirve XLSX a JSONL?",
    "GitHub de LLM StoryTeller",
    "Proyecto sobre predicción de fallos",
    "Nombre del TFM",
    "Breve descripción de MIDAS",
    "Herramientas de Fox-Detector"
]

# Preguntas difíciles (dificultad 1) - Requieren ANÁLISIS o EVALUACIÓN
prompts_diff1 = [
    # Evaluaciones profesionales
    "¿Encajaría Marcos en un puesto de liderazgo?",
    "¿Es adecuado para investigación en IA?",
    "¿Tiene suficiente experiencia para ser senior?",
    "Evalúa sus capacidades en desarrollo IA",
    "¿Sería un buen fit para un equipo de datos?",
    "¿Puede liderar proyectos técnicos?",
    "¿Por qué debería contratarlo?",
    "Valor que aportaría a un equipo ML",
    "¿Encaja con un rol de MLOps?",
    "¿Es buen candidato para fintech?",

    # Análisis comparativo y trayectoria
    "¿Cómo se compara con otros profesionales de IA?",
    "¿Qué lo hace destacar frente a perfiles similares?",
    "¿Es coherente su trayectoria profesional?",
    "Fortalezas de su perfil profesional",
    "Debilidades en su CV",
    "Evaluación de su transición hacia IA",
    "Habilidades destacables vs. perfil estándar",
    "Dirección profesional que parece seguir",
    "¿Es adecuada su formación para su experiencia?",
    "Relevancia de su experiencia para el campo IA",

    # Análisis técnico de proyectos
    "Decisiones de arquitectura críticas en MIDAS",
    "Mejoras posibles para ChatCV",
    "Escalabilidad de LLM StoryTeller",
    "Limitaciones del modelo Fox-Detector",
    "Base de datos vectorial de MIDAS architech",
    "Enfoque de prompting en MIDAS",
    "Optimización de HDD Failure ML",
    "Desafíos de implementación multiagente",
    "Innovación del enfoque RAG en ChatCV",
    "Mejoras para Gather-Tracker",
    "Hablame sobre MIDAS",

    # Proyección y recomendaciones
    "Certificaciones recomendadas para mejorar su perfil",
    "Roles futuros recomendados",
    "Áreas donde debería profundizar",
    "Especialización IA más adecuada",
    "Estrategias para mejor posicionamiento laboral",
    "Tecnologías emergentes que debería aprender",
    "Beneficios de especializarse en LLMs",
    "Tipo de empresa ideal para su desarrollo",
    "Áreas de mejora identificadas",
    "Aprovechamiento de experiencia como LLM Specialist"
]

# Preguntas no relacionadas (dificultad 2)
prompts_diff2 = [
    # Cocina y gastronomía
    "Receta tortilla de patatas",
    "Dame una receta de paella",
    "Secreto del buen gazpacho",
    "Platos vegetarianos sencillos",
    "Ingredientes auténticos carbonara",
    "Pan casero sin amasado",
    "Cómo hacer tiramisú",
    "Maridaje para pescado",
    "Arroz perfecto técnica",
    "Desayuno saludable rápido",

    # Entretenimiento y cultura
    "Series parecidas a Breaking Bad",
    "Mejores películas 2023",
    "Libros recomendados ciencia ficción",
    "Explicación final Inception",
    "Museos imprescindibles Madrid",
    "Historia del flamenco",
    "Podcasts interesantes 2024",
    "Autor de Cien años de soledad",
    "Obras famosas Picasso",
    "Videojuegos estrategia recomendados",

    # Ciencia y conocimientos generales
    "Por qué el cielo es azul",
    "Teoría relatividad explicación sencilla",
    "Funcionamiento vacunas ARNm",
    "Posibilidad vida otros planetas",
    "Qué es inteligencia artificial",
    "Cambio climático consecuencias",
    "Formación huracanes",
    "Significado sueños con agua",
    "Animal más inteligente tierra",
    "Efecto mariposa explicado",

    # Salud y bienestar
    "Ejercicios dolor espalda",
    "Mejorar calidad sueño",
    "Beneficios vitamina D",
    "Dieta mediterránea características",
    "Técnicas meditación principiantes",
    "Superar ansiedad social",
    "Rutina ejercicios casa",
    "Cantidad agua diaria recomendada",
    "Alimentos ricos hierro",
    "Consejos buena postura"
]

# Construcción del dataset equilibrado
datos = []
for prompt in prompts_diff0: datos.append({"prompt": prompt, "dificultad": 0})
for prompt in prompts_diff1: datos.append({"prompt": prompt, "dificultad": 1})
for prompt in prompts_diff2: datos.append({"prompt": prompt, "dificultad": 2})

# Creación y guardado del DataFrame
import pandas as pd
import random

random.shuffle(datos)
df = pd.DataFrame(datos)

print(f"Distribución del dataset:")
print(df['dificultad'].value_counts())

df.to_csv("dataset_raw.csv", index=False)
print("Dataset final creado y listo para producción.")

Distribución del dataset:
dificultad
1    41
2    40
0    40
Name: count, dtype: int64
Dataset final creado y listo para producción.


## 3. Preprocesar el texto

De esta forma no dará problemas al entrenarse.

In [3]:
# Celda 3: Preprocesamiento de texto
def clean_text(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

df = pd.read_csv("dataset_raw.csv")
df['cleaned_prompt'] = df['prompt'].apply(clean_text)
texts = df['cleaned_prompt'].tolist()
labels = df['dificultad'].tolist()

Dividimos el dataset en test y train

In [4]:
# Celda 4: División del dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42, shuffle=True
)

Y lo tokenizamos usando el tokenizador propio del modelo a utilizar. Con su longitud maxima de 512 tokens.

In [5]:
# Celda 5: Tokenización
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

Creamos los dataloaders (para procesar el dataset por lotes)

In [6]:
# Celda 6: Creación de DataLoaders
class MIDASDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MIDASDataset(train_encodings, train_labels)
test_dataset = MIDASDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

Configuramos el modelo y las labels...

In [7]:
# Celda 7: Configuración del modelo
model = BertForSequenceClassification.from_pretrained(
    'dccuchile/bert-base-spanish-wwm-cased',
    num_labels=3
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## 4. Entrenar el modelo

Aqui definimos los parametros del entrenamiento...

In [8]:
# Celda 8: Configuración del entrenamiento
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 10
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Y ahora entrenamos al modelo. Serán 10 epocas y se guardará aquel que obtenga un menor "loss" en el conjunto de test. De esta forma nos quedaremos con aquel que tenga un mejor desempeño sin caer en el sobreajuste.

In [9]:
# Celda 9: Entrenamiento
best_val_loss = float('inf')
best_model_state = None

for epoch in range(num_epochs):
    # Entrenamiento
    model.train()
    total_train_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        outputs = model(**inputs, labels=batch['labels'].to(device))
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)

    # Evaluación en el conjunto de test (o validación)
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            outputs = model(**inputs, labels=batch['labels'].to(device))
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(test_loader)

    print(f"Epoch {epoch+1} - Train Loss: {avg_train_loss:.4f} - Validation Loss: {avg_val_loss:.4f}")

    # Guardar el mejor modelo según el loss en el conjunto de validación
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = copy.deepcopy(model.state_dict())
        print(f"--> Mejor modelo actualizado en Epoch {epoch+1}")

# Cargar el estado del mejor modelo
model.load_state_dict(best_model_state)
print("Entrenamiento completado. Se cargó el modelo con mejor desempeño en validación.")


Epoch 1: 100%|██████████| 12/12 [00:05<00:00,  2.34it/s]


Epoch 1 - Train Loss: 1.1041 - Validation Loss: 1.0429
--> Mejor modelo actualizado en Epoch 1


Epoch 2: 100%|██████████| 12/12 [00:02<00:00,  5.76it/s]


Epoch 2 - Train Loss: 0.8875 - Validation Loss: 0.9033
--> Mejor modelo actualizado en Epoch 2


Epoch 3: 100%|██████████| 12/12 [00:00<00:00, 12.03it/s]


...

Epoch 8 - Train Loss: 0.0153 - Validation Loss: 0.3209


Epoch 9: 100%|██████████| 12/12 [00:00<00:00, 16.14it/s]


Epoch 9 - Train Loss: 0.0122 - Validation Loss: 0.2974


Epoch 10: 100%|██████████| 12/12 [00:00<00:00, 16.11it/s]

Epoch 10 - Train Loss: 0.0111 - Validation Loss: 0.2699
Entrenamiento completado. Se cargó el modelo con mejor desempeño en validación.


Vamos a sacar el reporte de clasificación para ver el desempeño

In [10]:
# Celda 10: Evaluación
model.eval()
predictions, true_labels = [], []

for batch in tqdm(test_loader, desc="Evaluando"):
    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(batch['labels'].cpu().numpy())

print("\nReporte de clasificación:")
print(classification_report(true_labels, predictions))

Evaluando: 100%|██████████| 4/4 [00:00<00:00, 46.92it/s]


Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       1.00      0.78      0.88         9
           2       1.00      1.00      1.00         8

    accuracy                           0.92        25
   macro avg       0.93      0.93      0.92        25
weighted avg       0.94      0.92      0.92        25



Muy buen resultado. Quizas un pelin bajo el recall de "pregunta dificil" (comparado con los otros), pero es MUY buen resultado.

Ahora, guardamos el modelo y el tokenizador.

In [11]:
# Celda 11: Guardar modelo
model.save_pretrained("prompt_analysis")
tokenizer.save_pretrained("prompt_analysis")

('prompt_analysis/tokenizer_config.json',
 'prompt_analysis/special_tokens_map.json',
 'prompt_analysis/vocab.txt',
 'prompt_analysis/added_tokens.json')

Y vamos a probar algunos ejemplos:

In [12]:
# Celda 12: Función de predicción
def clasificar_dificultad(texto):
    # Preprocesar
    texto_limpio = clean_text(texto)

    # Tokenizar
    inputs = tokenizer(texto_limpio, return_tensors="pt", truncation=True, max_length=512).to(device)

    # Predecir
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediccion = torch.argmax(logits, dim=1).item()

    # Mapeo de etiquetas descriptivas
    etiquetas = {
        0: "fácil",
        1: "difícil",
        2: "no relacionado"
    }

    return prediccion, etiquetas[prediccion]

# Ejemplo de uso
while True:
    user_input = input("\nIngrese su pregunta (o 'salir' para terminar): ")
    if user_input.lower() == 'salir':
        break

    dificultad_num, dificultad_texto = clasificar_dificultad(user_input)
    print(f"Dificultad clasificada: {dificultad_num} - {dificultad_texto}")


Ingrese su pregunta (o 'salir' para terminar): que sabes sobre marcos
Dificultad clasificada: 0 - fácil

Ingrese su pregunta (o 'salir' para terminar): que hizo marcos en sus practicas de grupo oro
Dificultad clasificada: 0 - fácil

Ingrese su pregunta (o 'salir' para terminar): en que consiste su tfm midas
Dificultad clasificada: 0 - fácil

Ingrese su pregunta (o 'salir' para terminar): hablame sobre como esta hecho midas architech
Dificultad clasificada: 1 - difícil

Ingrese su pregunta (o 'salir' para terminar): encajaria marcos para un puesto de liderazgo?
Dificultad clasificada: 1 - difícil

Ingrese su pregunta (o 'salir' para terminar): dime el sentido de la vida
Dificultad clasificada: 2 - no relacionado

Ingrese su pregunta (o 'salir' para terminar): salir


Estamos listos para integrarlo en nuestro ChatBot-CV.